In [5]:
import torch
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import random

# 生成数据

In [24]:
num_inputs = 2 # 特征数量
num_examples = 1000
true_w = [2,-3.4]
true_b = 4.2
features = torch.randn(num_examples,num_inputs,dtype=torch.float32)
labels = true_w[0] * features[:,0] + true_w[1] * features[:,1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()),
                       dtype=torch.float32)

1000

In [23]:
scatter = go.Scatter(x=features[:,1],y=labels,mode='markers')
fig = go.Figure(scatter)
fig.show()

In [35]:
def data_iter(batch_size,features,labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        j = torch.LongTensor(indices[i: min(i + batch_size, num_examples)])
        yield  features.index_select(0, j), labels.index_select(0, j)

In [39]:
for X,y in data_iter(10,features=features,labels=labels):
    print(X,y)f

tensor([[ 0.9672, -0.3857],
        [-1.8090,  0.3134],
        [-0.0338,  0.6842],
        [-0.4492,  0.6827],
        [-1.6243, -0.2449],
        [-0.3893,  0.9474],
        [ 0.7205,  1.0232],
        [ 0.3426,  2.1089],
        [ 1.1218, -0.5886],
        [-1.5580,  0.2021]]) tensor([ 7.4347, -0.4872,  1.8022,  0.9754,  1.7998,  0.1945,  2.1662, -2.2817,
         8.4377,  0.3970])
tensor([[ 0.2221,  1.4004],
        [-0.6219,  0.7646],
        [-0.5940,  0.4376],
        [ 0.3671,  0.6582],
        [-1.0113,  1.0405],
        [-0.3138,  0.1724],
        [-0.0143, -1.8417],
        [ 0.8333, -1.6837],
        [ 0.7878, -0.6112],
        [ 0.5108,  0.6862]]) tensor([-0.1042,  0.3442,  1.5146,  2.6865, -1.3664,  2.9849, 10.4482, 11.5929,
         7.8533,  2.8833])
tensor([[ 1.4168,  1.8180],
        [ 0.0883, -0.1772],
        [ 0.4557, -1.1087],
        [ 0.2836,  0.1319],
        [ 0.3282, -0.6561],
        [ 0.4407,  0.1276],
        [-0.5416,  0.3996],
        [-1.2314, -0.6043],


In [45]:
w = torch.tensor(np.random.normal(0,0.01,(num_inputs,1)),dtype=torch.float32)
b = torch.zeros(1,dtype=torch.float32)

In [48]:
w.requires_grad_(requires_grad=True)
b.requires_grad_(requires_grad=True)

tensor([0.], requires_grad=True)

In [49]:
def linreg(X,w,b):
    return torch.mm(X,w) + b

In [50]:
def squared_loss(y_hat,y):
    return (y_hat - y.view(y_hat.size())) ** 2 / 2

In [51]:
def sgd(params, lr, batch_size):
    for param in params:
        param.data -= lr * param.grad / batch_size

In [52]:
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

In [55]:
for epoch in range(num_epochs):
    for X,y in data_iter(batch_size=10,features=features,labels=labels):
        l = loss(net(X,w,b),y).sum() # 计算损失
        l.backward() # 反向传播
        sgd([w,b],lr,batch_size=10) # 随机梯度下降
        w.grad.data.zero_() 
        b.grad.data.zero_()
    train_l = loss(net(features,w,b),labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().item()))

epoch 1, loss 0.036332
epoch 2, loss 0.000143
epoch 3, loss 0.000047


In [56]:
print(true_w, '\n', w)
print(true_b, '\n', b)

[2, -3.4] 
 tensor([[ 1.9996],
        [-3.3996]], requires_grad=True)
4.2 
 tensor([4.1998], requires_grad=True)


# 此为手动实现，从原理的实现